TO DO

-remove pka values that are from residues not in the pdb file?

-figure out what to do when the pdb file doesnt exist (after figuring out where to store all of pkPDBs PDBs)

-make rtf download algorithm and handle the atypical residues 

-put cross checks to make sure this shit is correct,

-make visualization 


NOTES

-f(re: PKAD data) PDB ID 1lz3 was removed (from where? whatever) hence was manually downloaded from http://oca.weizmann.ac.il/oca-bin/ocashort?id=135L 

i had to manually change the pypka/pkPDB file from .csv.gz to .csv in my computer.

In [1]:
from Bio.PDB import PDBList
from Bio.PDB import *
import pandas as pd
import MDAnalysis as mda
import numpy as np
np.set_printoptions(threshold=np.inf) #1000 is default
import random
import math
from IPython.display import display, HTML

In [2]:
#PANDAS DATAFRAME PRODUCTION CELL FOR PKAD

def input_df(external_file_name):

#load the PKAD csv into pandas 
    df = pd.read_csv(external_file_name) 

#extract the needed info from the PKAD csv
    df = df[["PDB ID","Res ID","Res Name", "Expt. pKa"]]

#get rid of the N/A values (For Jesse: they are N/A because the CSV has an empty cell where the Res ID should be)
    df=df.dropna(ignore_index=True)

#make an identifier 
    ID=[]
    for i in range(len(df)):
        pdb=str.lower(df.iloc[i]['PDB ID'])
        res_n=str(math.trunc(df.iloc[i]['Res ID']))
        res=str(df.iloc[i]['Res Name'])
        ids="-".join([pdb,res,res_n]) #combine with a hyphen
        ID=np.append(ID,ids) #array with the unique IDs
    
#add identifier array to df
    col = 'ID' #column name
    df[col] = ID #adding array
    
    df = df.rename(columns={'Expt. pKa': 'pKa'})

    return df
#inputdf=input_df("/Users/jessihoernschemeyer/Desktop/Thesis/WT_pka.csv")
#display(HTML(inputdf.to_html()))

In [3]:
#pypka/pkPDB dataframe production

def input_pkPDB_df(csv_file):
    df = pd.read_csv(csv_file, nrows=100, delimiter=";") [["idcode","residue_name","residue_number", "pk"]]



#make an identifier 
    ID=[]
    for i in range(len(df)):
        pdb=str(df.iloc[i]['idcode'])
        res_n=str(df.iloc[i]['residue_number'])
        res=str(df.iloc[i]['residue_name'])
        ids="-".join([pdb,res,res_n]) #combine with a hyphen
        ID=np.append(ID,ids) #array with the unique IDs
    
#add identifier array to df
    col = 'ID' #column name
    df[col] = ID #adding array
    
    
#rename columns to match df from pkad 
    df = df.rename(columns={'idcode': 'PDB ID', 'residue_number': 'Res ID', 'residue_name': 'Res Name', 'residue_number': 'Res ID', 'pk': 'pKa'})

    
    return df    

In [6]:
#only run this cell if you want to download pdb files..

def download_pdbs(df):
#get an unordered list of the pdbs in order to download them
    matrix = df[df.columns[0]]
    PDB_list = matrix.tolist()
    PDB_list=list(set(PDB_list)) 


#download the pdbs
    pdbl = PDBList()
    for i in PDB_list:
        pdbl.retrieve_pdb_file(i,pdir='PDB',file_format = 'pdb')


download_pdbs(input_df("/Users/jessihoernschemeyer/Desktop/Thesis/WT_pka.csv"))

Structure exists: 'PDB/pdb1wla.ent' 
Structure exists: 'PDB/pdb7rsa.ent' 
Structure exists: 'PDB/pdb1fw7.ent' 
Structure exists: 'PDB/pdb2mb5.ent' 
Structure exists: 'PDB/pdb1xnb.ent' 
Structure exists: 'PDB/pdb1hv1.ent' 
Structure exists: 'PDB/pdb1poh.ent' 
Structure exists: 'PDB/pdb1ypi.ent' 
Structure exists: 'PDB/pdb1ptd.ent' 
Structure exists: 'PDB/pdb9rnt.ent' 
Structure exists: 'PDB/pdb1rnz.ent' 
Structure exists: 'PDB/pdb3ssi.ent' 
Structure exists: 'PDB/pdb1jbb.ent' 
Structure exists: 'PDB/pdb3trx.ent' 
Structure exists: 'PDB/pdb1gb1.ent' 
Structure exists: 'PDB/pdb1qlp.ent' 
Structure exists: 'PDB/pdb1trs.ent' 
Structure exists: 'PDB/pdb1the.ent' 
Structure exists: 'PDB/pdb1a6m.ent' 
Structure exists: 'PDB/pdb1epg.ent' 
Structure exists: 'PDB/pdb2sni.ent' 
Structure exists: 'PDB/pdb1bi6.ent' 
Structure exists: 'PDB/pdb1dg9.ent' 
Structure exists: 'PDB/pdb1i0e.ent' 
Structure exists: 'PDB/pdb1qh7.ent' 
Structure exists: 'PDB/pdb1hng.ent' 
Structure exists: 'PDB/pdb1a91.ent' 
S

In [6]:
def mini_batches(input_df):
    #this function generates the atom information about the neighborhood of a titratable residue which is retrieved
    #from inputted data. It then adds the pdb name and pka value of the titratable residue for which the atoms belong
    #to, to the new dataframe, which becomes the new main dataframe. 
    #each new df made in this function corresponds to 1 row on the original/input/old df.
    
#get pdb name from input df to put into mda.
    pdb_res=input_df #just me being lazy, ignore..
    
    df_list=[]
    
    for i in range(1): #range(len(pdb_res))
        x=[]
        y=[]
        z=[]
    
        pdbname=str.lower(pdb_res.iloc[i]['PDB ID']) #zooming into pdb name
        #For Jesse: file name needs manual change of everything preceding {} for code to work
        pdb_file = "/Users/jessihoernschemeyer/PDB/pdb{}.ent".format(pdbname) 
    
#make mda universe with the pdb
        u = mda.Universe(pdb_file)

#get resids in format for mda to read.

        if bool(isinstance(pdb_res.iloc[1]['Res ID'], float) == True): #because pkPDB is already a truncated integer. 
            val=math.trunc(pdb_res.iloc[i]['Res ID']) 
        else:
            val=pdb_res.iloc[i]['Res ID'] 
        
        
        resid_in="resid {}".format(val) #string in string


#zooming into residue of 1 pdb: selects the atoms of the residue given by res id above and makes an atom group.
        res_atoms=u.select_atoms(resid_in) #makes an mda atom group. ORDER NOT PRESERVED AND DUPLICATES REMOVED FOR AGs

#get atoms in distance of residue we are focusing on
        string_nahe='around 10 {}'.format(resid_in) #all atoms around the word after the number, which is the cutoff distance
        nahe=u.select_atoms(string_nahe) #atom group 

#make a residue group of residues which are within the atom group of atoms in the vicinity of the t. residue.
        residues = nahe.residues #vicinity residues
    
#get the vicinity residues' names 
        res_name = residues.atoms.resnames
    
#get the vicinity residues' residue number
        res_ids = residues.atoms.resids
    
#get atom types of atoms in (each) vicinity residue  
        atomtype = residues.atoms.names
    
#get the positions of atoms in (each) vicinity residue
        positions=residues.atoms.positions

#seperate them into x, y, and z coordinate arrays for easier processing into the df
        for j in range(len(positions)):

    #x coordinate, 1 atom, 1 residue
            x=np.append(x,positions[j,0]) #goes through each atom of 1 pdb file and takes the x coordinate. 
            
        
        #pdb_res[column_name_x] = x #adds the x position as a column to the df.
    
    #y coordinate

            y=np.append(y,positions[j,1])
        #pdb_res[column_name_y] = y

    #z coordinate

            z=np.append(z,positions[j,2])
        
#get pka values
        pka = pdb_res.iloc[i]['pKa']

#make a dataframe of the neighborhood information
        df = pd.DataFrame({"PDB": pdbname,
                    "Res Name": res_name,
                    "Res No": res_ids,
                    "pKa": pka,
                    "Atom Type": atomtype,
                    "x": x,
                    "y":y,
                    "z":z
                          })

#name each df (mini batch) with the name of the unique identifier from the imported df.
        df.name = str((pdb_res.iloc[i]['ID']))

#makes the index column name = the identifer for which the neighborhood information belongs to    
        df.index.name = df.name
    
#add each df, which represents the information about the neighborhood of the titratable residue, to a list
        df_list.append(df) #append a list
    
#prepare residue name list for eventual export...
        resnames_set=list(set(res_name))
        
#TODO: make a check such that the number of rows in original input data frame = the length of df_list.  
    
        
    return df_list, resnames_set #df_list is a list of all the mini batches / new data frames. 

In [7]:
#pkPDB
input_df_pkPDB=input_pkPDB_df("/Users/jessihoernschemeyer/Downloads/pkas.csv")

#for Jesse (& Jessi) to download 1 pdb
pdbl = PDBList()
pdbl.retrieve_pdb_file("2bb7",pdir='PDB',file_format = 'pdb')

#run the function for the data from pkPDB
df_pkPDB, resnames_pkPDB = mini_batches(input_df_pkPDB) 

#display first (only) mini batch
df_pkPDB[0]

#optional: unhash to see full table
#display(HTML(df_pkPDB[0].to_html()))

Structure exists: 'PDB/pdb2bb7.ent' 


/Users/jessihoernschemeyer/miniconda3/envs/schnet/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: MN
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


,PDB,Res Name,Res No,pKa,Atom Type,x,y,z
2bb7-CTR-262,,,,,,,,
0,2bb7,ALA,58,2.49605,N,-3.365000,9.414,9.558
1,2bb7,ALA,58,2.49605,CA,-3.788000,8.629,8.399
2,2bb7,ALA,58,2.49605,C,-4.696000,7.457,8.775
3,2bb7,ALA,58,2.49605,O,-5.498000,7.024,7.958
4,2bb7,ALA,58,2.49605,CB,-2.577000,8.141,7.597
...,...,...,...,...,...,...,...,...
333,2bb7,HOH,461,2.49605,O,23.479000,0.083,-5.550
334,2bb7,HOH,478,2.49605,O,23.667999,-6.478,8.919
335,2bb7,HOH,506,2.49605,O,-4.700000,7.858,3.518


In [8]:
#PKAD
input_df_PKAD=input_df("/Users/jessihoernschemeyer/Desktop/Thesis/WT_pka.csv")

#for Jesse to download 1 pdb
pdbl = PDBList()
pdbl.retrieve_pdb_file("1a2p",pdir='PDB',file_format = 'pdb')

df_PKAD, resnames_PKAD = mini_batches(input_df_PKAD)

df_PKAD[0]
display(HTML(df_PKAD[0].to_html()))

Structure exists: 'PDB/pdb1a2p.ent' 


,PDB,Res Name,Res No,pKa,Atom Type,x,y,z
1a2p-ASP-8,,,,,,,,
0,1a2p,VAL,3,3.1,N,16.875000,37.901001,43.478001
1,1a2p,VAL,3,3.1,CA,17.997000,38.577000,42.794998
2,1a2p,VAL,3,3.1,C,18.146999,39.997002,43.308998
3,1a2p,VAL,3,3.1,O,17.180000,40.763000,43.280998
4,1a2p,VAL,3,3.1,CB,17.785999,38.592999,41.278999
5,1a2p,VAL,3,3.1,CG1,18.621000,39.605999,40.573002
6,1a2p,VAL,3,3.1,CG2,18.024000,37.189999,40.695999
7,1a2p,ILE,4,3.1,N,19.355000,40.376999,43.689999
8,1a2p,ILE,4,3.1,CA,19.568001,41.805000,44.109001


In [ ]:
#junk cell / ignore 


#make unique identifier 
    data=pdb_res.to_numpy() #pandas to numpy
    pdb=[]
    res=[]
    randomm=[]
    rand=random.sample(range(1000,9999), len(data))
    ID=[]

    for i in range(len(data)):
        pdb=np.append(pdb,str(data[i,0]))
        res=np.append(res,str(math.trunc(data[i,1])))
        randomm=np.append(randomm,str(rand[i])) #random 4 digit number 
        ids="-".join([pdb[i],res[i],randomm[i]]) #combine with a hyphen
        ID=np.append(ID,ids) #array with the unique IDs
    
#add the unique ID to the dataframe 
    varA = 'Unique ID' #column name
    pdb_res[varA] = ID #adding the array i wanna add to df
    pdb_res